# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13565

In [5]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [6]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [7]:
sents_tokenize = [tokenizer.tokenize(sent) for sent in sents]
sents_tokenize[4:6]

[['и',
  'каждый',
  'раз',
  'молодой',
  'человек',
  'проходя',
  'мимо',
  'чувствовал',
  'какое',
  'то',
  'болезненное',
  'и',
  'трусливое',
  'ощущение',
  'которого',
  'стыдился',
  'и',
  'от',
  'которого',
  'морщился'],
 ['он',
  'был',
  'должен',
  'кругом',
  'хозяйке',
  'и',
  'боялся',
  'с',
  'нею',
  'встретиться']]

In [8]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [9]:
len(words) # всего слов

173403

In [10]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [11]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [12]:
sents_tokenize = [[word for word in text_cur if word not in stopwords_ru] for text_cur in sents_tokenize]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Каждый должен знать основы нейронных сетей!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'каждый должный знать основа нейронный сеть'

In [15]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Обработаем все слова из датасета. 

In [17]:
sents_tokenize = [[morph.normal_forms(word)[0] for word in text_cur] for text_cur in sents_tokenize]

In [18]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [19]:
len(words) # всего слов

93069

In [20]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [21]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [23]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `vector_size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [25]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

CPU times: user 11.7 s, sys: 124 ms, total: 11.8 s
Wall time: 4.28 s


(7945071, 9306900)

In [26]:
model.corpus_count # число примеров в обучающей выборке

13565

## 3. Свойства модели

In [33]:
# вектор слова
model.wv['старуха'][:10]

array([ 0.4280811 ,  1.0806488 , -0.3607423 , -1.104609  ,  0.3137776 ,
       -1.6037107 , -2.846244  , -0.9434775 , -0.7211046 ,  0.17917341],
      dtype=float32)

In [34]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [35]:
# похожести слов 
model.wv.similarity('тварь', 'право')

0.06303523

In [36]:
model.wv.similarity('старуха', 'топор')

0.3821662

In [37]:
model.wv.similarity('тварь', 'тварь')

0.99999994

In [38]:
# самые похожие
model.wv.most_similar('старуха')

[('лизавета', 0.49433615803718567),
 ('чиновница', 0.4205626845359802),
 ('близко', 0.4189166724681854),
 ('наклепать', 0.4070381224155426),
 ('разыскать', 0.4046613276004791),
 ('ограбить', 0.38955026865005493),
 ('обратно', 0.38676783442497253),
 ('топор', 0.38216617703437805),
 ('обух', 0.3689005672931671),
 ('серьга', 0.36871662735939026)]

In [39]:
model.wv.most_similar('лестница')

[('ступенька', 0.528121292591095),
 ('всходить', 0.5243401527404785),
 ('прихожая', 0.5034546852111816),
 ('передний', 0.47807765007019043),
 ('вниз', 0.46348679065704346),
 ('ступень', 0.4568527936935425),
 ('узенький', 0.4504789113998413),
 ('спускаться', 0.450210839509964),
 ('цыпочки', 0.44844818115234375),
 ('прислуга', 0.42274972796440125)]

In [41]:
# арифметика
model.wv.most_similar(positive=['раскольников'], negative=['тварь'])[:10]

[('хорошенько', 0.3037472069263458),
 ('городов', 0.2816304862499237),
 ('папироска', 0.279782235622406),
 ('великолепно', 0.2763938009738922),
 ('несколько', 0.26956114172935486),
 ('затем', 0.26229575276374817),
 ('скрываться', 0.255309522151947),
 ('приятно', 0.2508450746536255),
 ('приближаться', 0.24801000952720642),
 ('казённый', 0.24088609218597412)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [42]:
model_path = "./our_w2v.model"
model.save(model_path)

In [43]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [44]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [45]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]


In [46]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [47]:
len(sents_tokenize2)

254

Дополняем модель.

In [48]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

(177193, 269000)

In [50]:
our_model.wv.most_similar('ядро')

[('изумруд', 0.681205689907074),
 ('скатерть', 0.629682183265686),
 ('грызть', 0.5863677263259888),
 ('чистый', 0.5801365971565247),
 ('журчать', 0.5591646432876587),
 ('орешек', 0.5571174621582031),
 ('пристань', 0.551948606967926),
 ('золотой', 0.5428691506385803),
 ('научить', 0.5405980944633484),
 ('злато', 0.529037356376648)]

Пример со старым словом.

In [51]:
our_model.wv.most_similar('сын')

[('губерния', 0.5759544372558594),
 ('обеспечить', 0.497567355632782),
 ('благословить', 0.4963618516921997),
 ('расспрос', 0.49000808596611023),
 ('уезд', 0.48182547092437744),
 ('враньё', 0.4693182408809662),
 ('особо', 0.4667699337005615),
 ('родственница', 0.46160581707954407),
 ('наущение', 0.4599708914756775),
 ('адвокатский', 0.45724308490753174)]

In [52]:
model.wv.most_similar('сын')

[('детство', 0.49924013018608093),
 ('расспрос', 0.47770294547080994),
 ('поступок', 0.4652251601219177),
 ('губерния', 0.46455374360084534),
 ('наущение', 0.45484694838523865),
 ('секрет', 0.44692593812942505),
 ('приём', 0.4465712606906891),
 ('джентльмен', 0.434931218624115),
 ('обеспечить', 0.4348464012145996),
 ('раздражать', 0.4335572421550751)]